In [2]:
from keras.models import Sequential
from keras.layers import Dense, Activation, LeakyReLU, BatchNormalization
from keras import backend
from keras.callbacks import TensorBoard
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import warnings
import math
warnings.filterwarnings("ignore")

In [3]:
# Hyperparams
n_units = 400
layers = 4
n_batch = 4096
n_epochs = 50

In [4]:
import pennylane as qml

n_qubits = 5
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def qnode(inputs, weights):
    qml.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.BasicEntanglerLayers(weights, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(wires=i)) for i in range(n_qubits)]

In [ ]:
"""my_bucket = f"amazon-braket-exp"
my_prefix = "piyush"
s3_folder = (my_bucket, my_prefix)
device_arn = "arn:aws:braket:::device/quantum-simulator/amazon/sv1"
dev = qml.device('braket.aws.qubit', device_arn=device_arn, wires=n_qubits, s3_destination_folder=s3_folder)"""

In [5]:
n_layers = 6
weight_shapes = {"weights": (n_layers, n_qubits)}
qlayer = qml.qnn.KerasLayer(qnode, weight_shapes, output_dim=n_qubits)

In [6]:
call_df = pd.read_csv('data.csv')

In [7]:
call_df.head()

,strike_price,best_bid,best_offer,date_ndiff,treasury_rate,closing_price,sigma_20
0,334.0,4.50,4.57,3,15.146374,333.84,0.3812
1,335.0,3.83,3.90,3,15.146374,333.84,0.3647
2,336.0,3.24,3.28,3,15.146374,333.84,0.3500
3,337.0,2.68,2.72,3,15.146374,333.84,0.3358
4,338.0,2.19,2.22,3,15.146374,333.84,0.3238


In [8]:
call_X_train, call_X_test, call_y_train, call_y_test = train_test_split(call_df.drop(['best_bid', 'best_offer'], axis=1),
                                                                        call_df[['best_bid', 'best_offer']],
                                                                        test_size=0.1, random_state=42)

In [9]:
model = Sequential()
model.add(Dense(n_units, input_dim=call_X_train.shape[1]))
model.add(LeakyReLU())

for _ in range(layers - 1):
    model.add(Dense(n_units))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
model.add(Dense(n_qubits, activation='relu'))
model.add(qlayer)
model.add(Dense(2, activation='relu'))
model.compile(loss='mse', optimizer=Adam())

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 400)               2400      
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 400)               0         
                                                                 
 dense_1 (Dense)             (None, 400)               160400    
                                                                 
 batch_normalization (Batch  (None, 400)               1600      
 Normalization)                                                  
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 400)               0         
                                                                 
 dense_2 (Dense)             (None, 400)               160400    
                                                        

In [12]:
history = model.fit(call_X_train, call_y_train, 
                    batch_size=n_batch, epochs=n_epochs, 
                    validation_split = 0.01,
                    callbacks=[TensorBoard()],
                    verbose=1)

Epoch 1/50
 3/10 [========>.....................] - ETA: 7s - loss: 665.7378WARNING:tensorflow:You are casting an input of type complex128 to an incompatible dtype float32.  This will discard the imaginary part and may not be what you intended.


KeyboardInterrupt: 

In [ ]:
model.save('mlp2-call30.h5')

In [ ]:
call_y_pred30 = model.predict(call_X_test)

In [ ]:
print('equilibrium mse', np.mean(np.square(np.mean(call_y_test.values, axis=1) - np.mean(call_y_pred30, axis=1))))
print('spread mse', np.mean(np.square(np.diff(call_y_test.values, axis=1) - np.diff(call_y_pred30, axis=1))))

In [ ]:
model.compile(loss='mse', optimizer=Adam(1e-4))
history = model.fit(call_X_train, call_y_train, 
                    batch_size=n_batch, epochs=n_epochs, 
                    validation_split = 0.01,
                    callbacks=[TensorBoard()],
                    verbose=1)

In [ ]:
model.save('mlp2-call40.h5')

In [ ]:
call_y_pred40 = model.predict(call_X_test)
print('equilibrium mse', np.mean(np.square(np.mean(call_y_test.values, axis=1) - np.mean(call_y_pred40, axis=1))))
print('spread mse', np.mean(np.square(np.diff(call_y_test.values, axis=1) - np.diff(call_y_pred40, axis=1))))

In [ ]:
model.compile(loss='mse', optimizer=Adam(1e-5))
history = model.fit(call_X_train, call_y_train, 
                    batch_size=n_batch, epochs=10, 
                    validation_split = 0.01,
                    callbacks=[TensorBoard()],
                    verbose=1)

In [ ]:
model.save('mlp2-call50.h5')
call_y_pred50 = model.predict(call_X_test)
print('equilibrium mse', np.mean(np.square(np.mean(call_y_test.values, axis=1) - np.mean(call_y_pred50, axis=1))))
print('spread mse', np.mean(np.square(np.diff(call_y_test.values, axis=1) - np.diff(call_y_pred50, axis=1))))

In [ ]:
model.compile(loss='mse', optimizer=Adam(1e-6))
history = model.fit(call_X_train, call_y_train, 
                    batch_size=n_batch, epochs=10, 
                    validation_split = 0.01,
                    callbacks=[TensorBoard()],
                    verbose=1)

In [ ]:
model.save('mlp2-call60.h5')
call_y_pred60 = model.predict(call_X_test)
print('equilibrium mse', np.mean(np.square(np.mean(call_y_test.values, axis=1) - np.mean(call_y_pred60, axis=1))))
print('spread mse', np.mean(np.square(np.diff(call_y_test.values, axis=1) - np.diff(call_y_pred60, axis=1))))